**Title**: RECOMMENDER SYSTEM

**Description**: Here we aim to run Content based recommendation model on the dataset to recommend a candidate to the skills they require to migrate to their desired career

**Author**: Benedict Ibe

**Date Created**: 30/07/2023

In [90]:
import nltk

StatementMeta(Nurture, 1296, 2, Finished, Available)

In [91]:
%%pyspark

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
candidate_skilllevel= spark.sql("SELECT \
Id                                          as Cand_ID, \
recruit_candidatecontact                    as Candidate_contactno, \
crimson_proficiency                         as Cand_proficiency, \
crimson_requirement                         as Cand_requirement, \
crimson_experience                          as Cand_experience, \
crimson_experienceperiod                    as Cand_exp_period, \
crimson_skill                               as Cand_skill, \
crimson_name                                as Cand_name, \
crimson_skilllevelid                        as Cand_skilllevel_ID, \
lower(crimson_skillname)                    as Cand_skill_name, \
crimson_level                               as Cand_level \
FROM dataverse_edensmithcon_org87f26120.crimson_skilllevel")

StatementMeta(Nurture, 1296, 3, Finished, Available)

In [92]:
candidate_skilllevel.show()

StatementMeta(Nurture, 1296, 4, Finished, Available)

+--------------------+--------------------+----------------+----------------+---------------+---------------+--------------------+---------+--------------------+--------------------+----------+
|             Cand_ID| Candidate_contactno|Cand_proficiency|Cand_requirement|Cand_experience|Cand_exp_period|          Cand_skill|Cand_name|  Cand_skilllevel_ID|     Cand_skill_name|Cand_level|
+--------------------+--------------------+----------------+----------------+---------------+---------------+--------------------+---------+--------------------+--------------------+----------+
|ab1e9a1d-104e-eb1...|03c6701e-104e-eb1...|       143570003|       143570000|             43|      143570000|089ac1e6-27bd-ea1...|     null|ab1e9a1d-104e-eb1...|                gdpr|        97|
|ae1e9a1d-104e-eb1...|03c6701e-104e-eb1...|       143570003|       143570000|             43|      143570000|9f9fc31c-8a59-e11...|     null|ae1e9a1d-104e-eb1...|    data warehousing|        99|
|b51e9a1d-104e-eb1...|03c6701e

In [93]:
%%pyspark

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
candidate_workhist =spark.sql("SELECT \
Id                                  as ID , \
recruit_candidatecontact            as Candidate_contact,\
crimson_startdate                   as Candidate_workstrt_date,\
crimson_enddate                     as Candidate_workend_date, \
crimson_workhistoryid               as WorkHistory_ID,\
lower(crimson_description)          as Candidate_work_description,\
crimson_jobtitle                    as Candidate_job_title ,\
crimson_name                        as Cand_workplace \
FROM dataverse_edensmithcon_org87f26120.crimson_workhistory")

StatementMeta(Nurture, 1296, 5, Finished, Available)

In [94]:
from pyspark.sql.functions import to_date

StatementMeta(Nurture, 1296, 6, Finished, Available)

In [95]:
#Here we convert the date-time format to date
df_hist = candidate_workhist.withColumn("Cnd_start_date", to_date("Candidate_workstrt_date"))
df_hist.show()

StatementMeta(Nurture, 1296, 7, Finished, Available)

+--------------------+--------------------+-----------------------+----------------------+--------------------+--------------------------+--------------------+--------------------+--------------+
|                  ID|   Candidate_contact|Candidate_workstrt_date|Candidate_workend_date|      WorkHistory_ID|Candidate_work_description| Candidate_job_title|      Cand_workplace|Cnd_start_date|
+--------------------+--------------------+-----------------------+----------------------+--------------------+--------------------------+--------------------+--------------------+--------------+
|6ae7162b-104e-eb1...|14c6701e-104e-eb1...|    2019-05-01 00:00:00|   2020-08-01 00:00:00|6ae7162b-104e-eb1...|      ; lendinvest - --...|        Data analyst|Data analyst - Le...|    2019-05-01|
|44e4bc29-104e-eb1...|03c6701e-104e-eb1...|    2017-01-01 00:00:00|                  null|44e4bc29-104e-eb1...|      key achievements\...|Reporting and Dat...|Reporting and Dat...|    2017-01-01|
|71e7162b-104e-eb1..

In [96]:
#Here we convert the date-time format to date
df_hist2 = df_hist.withColumn("Cnd_end_date", to_date("Candidate_workend_date"))
df_hist2.show()

StatementMeta(Nurture, 1296, 8, Finished, Available)

+--------------------+--------------------+-----------------------+----------------------+--------------------+--------------------------+--------------------+--------------------+--------------+------------+
|                  ID|   Candidate_contact|Candidate_workstrt_date|Candidate_workend_date|      WorkHistory_ID|Candidate_work_description| Candidate_job_title|      Cand_workplace|Cnd_start_date|Cnd_end_date|
+--------------------+--------------------+-----------------------+----------------------+--------------------+--------------------------+--------------------+--------------------+--------------+------------+
|6ae7162b-104e-eb1...|14c6701e-104e-eb1...|    2019-05-01 00:00:00|   2020-08-01 00:00:00|6ae7162b-104e-eb1...|      ; lendinvest - --...|        Data analyst|Data analyst - Le...|    2019-05-01|  2020-08-01|
|44e4bc29-104e-eb1...|03c6701e-104e-eb1...|    2017-01-01 00:00:00|                  null|44e4bc29-104e-eb1...|      key achievements\...|Reporting and Dat...|Repor

In [97]:
#Here we select the required columns
df_hist3 = df_hist2.select('ID', 'WorkHistory_ID', 'Candidate_contact', 'Candidate_job_title', 'Cnd_start_date', 'Cnd_end_date')
df_hist3.show()

StatementMeta(Nurture, 1296, 9, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|                  ID|      WorkHistory_ID|   Candidate_contact| Candidate_job_title|Cnd_start_date|Cnd_end_date|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+
|6ae7162b-104e-eb1...|6ae7162b-104e-eb1...|14c6701e-104e-eb1...|        Data analyst|    2019-05-01|  2020-08-01|
|44e4bc29-104e-eb1...|44e4bc29-104e-eb1...|03c6701e-104e-eb1...|Reporting and Dat...|    2017-01-01|        null|
|71e7162b-104e-eb1...|71e7162b-104e-eb1...|14c6701e-104e-eb1...|             analyst|    2018-04-01|  2019-05-01|
|76e7162b-104e-eb1...|76e7162b-104e-eb1...|03c6701e-104e-eb1...|Data and Process ...|    2015-07-01|  2017-01-01|
|53e4bc29-104e-eb1...|53e4bc29-104e-eb1...|14c6701e-104e-eb1...|           associate|    2017-05-01|  2018-03-01|
|5de4bc29-104e-eb1...|5de4bc29-104e-eb1...|14c6701e-104e-eb1...|Risk Analyst / Ri...|   

In [98]:
#Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import datediff, floor
from pyspark.sql.functions import lit

StatementMeta(Nurture, 1296, 10, Finished, Available)

GETTING THE NUMBER OF MONTHS BETWEEN THE START TIME AND THE END TIME

In [99]:
#GETTING THE NUMBER OF MONTHS BETWEEN THE START TIME AND THE END TIME
df_hist4 = df_hist3.withColumn("Duration", floor((datediff(df_hist3["Cnd_end_date"], df_hist3["Cnd_start_date"]) / 7)/4))
df_hist4.show()

StatementMeta(Nurture, 1296, 11, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+
|                  ID|      WorkHistory_ID|   Candidate_contact| Candidate_job_title|Cnd_start_date|Cnd_end_date|Duration|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+
|6ae7162b-104e-eb1...|6ae7162b-104e-eb1...|14c6701e-104e-eb1...|        Data analyst|    2019-05-01|  2020-08-01|      16|
|44e4bc29-104e-eb1...|44e4bc29-104e-eb1...|03c6701e-104e-eb1...|Reporting and Dat...|    2017-01-01|        null|    null|
|71e7162b-104e-eb1...|71e7162b-104e-eb1...|14c6701e-104e-eb1...|             analyst|    2018-04-01|  2019-05-01|      14|
|76e7162b-104e-eb1...|76e7162b-104e-eb1...|03c6701e-104e-eb1...|Data and Process ...|    2015-07-01|  2017-01-01|      19|
|53e4bc29-104e-eb1...|53e4bc29-104e-eb1...|14c6701e-104e-eb1...|           associate|    2017-05-01|  2018-03-01|      10|
|5de4bc29-104e-e

DEFINE THE PARAMETER TO SEARCH

In [100]:
job_titleSearch = 'data'

StatementMeta(Nurture, 1296, 12, Finished, Available)

SEARCH FOR THE WILD CHARACTER DEFINED IN THE PARAMETER

In [101]:
#SEARCH FOR THE WILD CHARACTER DEFINED IN THE PARAMETER
df_hist5 = df_hist4.filter(df_hist4.Candidate_job_title.rlike(job_titleSearch)).sort(df_hist4.Candidate_job_title.asc())
df_hist5.show(50)

StatementMeta(Nurture, 1296, 13, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+
|                  ID|      WorkHistory_ID|   Candidate_contact| Candidate_job_title|Cnd_start_date|Cnd_end_date|Duration|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+
|085adb9a-96f8-ec1...|085adb9a-96f8-ec1...|d40b2dbe-8b98-ec1...|AML/KYC data Anal...|    2008-04-01|  2009-06-01|      15|
|833aaae0-b66c-ed1...|833aaae0-b66c-ed1...|2d66597b-a5c3-402...|Analyst / first d...|    2007-09-01|  2010-09-01|      39|
|d87123bc-6139-eb1...|d87123bc-6139-eb1...|e43b09ae-6139-eb1...|Analyst / first d...|    2007-09-01|  2010-09-01|      39|
|6f6396f1-6161-eb1...|6f6396f1-6161-eb1...|dfc02cd4-6161-eb1...|Architecture, Des...|    2016-02-01|  2017-03-01|      14|
|4f912272-8931-ec1...|4f912272-8931-ec1...|8c338153-8931-ec1...|Associate data sc...|    2018-05-01|  2018-09-01|       4|
|6e4476af-ab5c-e

In [102]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, lower
from pyspark.ml.feature import Tokenizer, Word2Vec
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

StatementMeta(Nurture, 1296, 14, Finished, Available)

REMOVE SPECIAL CHARACTERS AS A PREPROCESSING STEP

In [103]:
df_clean = df_hist4.withColumn("Candidate_job_title", lower(regexp_replace("Candidate_job_title", "[^a-zA-Z0-9\\s]", "")))
df_clean.show()

StatementMeta(Nurture, 1296, 15, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+
|                  ID|      WorkHistory_ID|   Candidate_contact| Candidate_job_title|Cnd_start_date|Cnd_end_date|Duration|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+
|6ae7162b-104e-eb1...|6ae7162b-104e-eb1...|14c6701e-104e-eb1...|        data analyst|    2019-05-01|  2020-08-01|      16|
|44e4bc29-104e-eb1...|44e4bc29-104e-eb1...|03c6701e-104e-eb1...|reporting and dat...|    2017-01-01|        null|    null|
|71e7162b-104e-eb1...|71e7162b-104e-eb1...|14c6701e-104e-eb1...|             analyst|    2018-04-01|  2019-05-01|      14|
|76e7162b-104e-eb1...|76e7162b-104e-eb1...|03c6701e-104e-eb1...|data and process ...|    2015-07-01|  2017-01-01|      19|
|53e4bc29-104e-eb1...|53e4bc29-104e-eb1...|14c6701e-104e-eb1...|           associate|    2017-05-01|  2018-03-01|      10|
|5de4bc29-104e-e

TREAT NULL VALUES

In [104]:
#Treat null values in the Candidate_job_title
df_clean2 = df_clean.na.drop(subset=["Candidate_job_title"])

StatementMeta(Nurture, 1296, 16, Finished, Available)

LOAD THE SAMPLE DATA TO BE USED FOR TRAINING

In [105]:
%%pyspark
df_salaries = spark.read.load('abfss://landing@esgadls2.dfs.core.windows.net/salaries_clean.csv', format='csv'
,header=True
)
df_salaries.show()

StatementMeta(Nurture, 1296, 17, Finished, Available)

+-----+---------+--------------------+------------------+--------------+----------------+-----------------+------------------+--------------------+------------------+--------------+----------------------+-------------------------+---------------+-------------+------------+-----------------+--------------------+-------------+
|index|salary_id|       employer_name|     location_name|location_state|location_country|location_latitude|location_longitude|           job_title|job_title_category|job_title_rank|total_experience_years|employer_experience_years|annual_base_pay|signing_bonus|annual_bonus|stock_value_bonus|            comments| submitted_at|
+-----+---------+--------------------+------------------+--------------+----------------+-----------------+------------------+--------------------+------------------+--------------+----------------------+-------------------------+---------------+-------------+------------+-----------------+--------------------+-------------+
|    0|        1|  

In [106]:
df_train = df_salaries.select('job_title', 'job_title_category')
df_train.show()

StatementMeta(Nurture, 1296, 18, Finished, Available)

+--------------------+------------------+
|           job_title|job_title_category|
+--------------------+------------------+
|    systems engineer|       Engineering|
|    senior developer|          Software|
|   software engineer|          Software|
|                 mts|             Other|
|   software engineer|          Software|
|       it technician|             Other|
|    lead interviewer|             Other|
|senior software e...|          Software|
|   software engineer|          Software|
|infrastructure/pl...|       Engineering|
|senior software e...|          Software|
|software develope...|          Software|
|       web developer|               Web|
|             analyst|              Data|
|             analyst|              Data|
|  it project manager|        Management|
|     senior engineer|       Engineering|
|member of technic...|             Other|
|student intern co-op|             Other|
|full stack developer|               Web|
+--------------------+------------

In [107]:
from pyspark.sql.functions import col, when

StatementMeta(Nurture, 1296, 19, Finished, Available)

In [108]:
#Treat null values in the job title column
dfTrain_clean2 = df_train.na.drop(subset=["job_title"])

StatementMeta(Nurture, 1296, 20, Finished, Available)

In [109]:
#prepare the dataset by converting cell to its appropriate general form
df_train2 = dfTrain_clean2.withColumn("job_title_category",
                   when(col("job_title").rlike("analyst"), "Data Analyst")
                   .when(col("job_title").rlike("data engineer"), "Data Engineer")
                   .when(col("job_title").rlike("data architect"), "Data Architect")
                   .when(col("job_title").rlike("data scientist"), "Data Science")
                   .when(col("job_title").rlike("database"), "Data Management")
                   .when(col("job_title").rlike("visualization"), "Data Analyst")
                   .otherwise(col("job_title_category"))
                  )

StatementMeta(Nurture, 1296, 21, Finished, Available)

In [110]:
df_train2.show(200)

StatementMeta(Nurture, 1296, 22, Finished, Available)

+--------------------+------------------+
|           job_title|job_title_category|
+--------------------+------------------+
|    systems engineer|       Engineering|
|    senior developer|          Software|
|   software engineer|          Software|
|                 mts|             Other|
|   software engineer|          Software|
|       it technician|             Other|
|    lead interviewer|             Other|
|senior software e...|          Software|
|   software engineer|          Software|
|infrastructure/pl...|       Engineering|
|senior software e...|          Software|
|software develope...|          Software|
|       web developer|               Web|
|             analyst|      Data Analyst|
|             analyst|      Data Analyst|
|  it project manager|        Management|
|     senior engineer|       Engineering|
|member of technic...|             Other|
|student intern co-op|             Other|
|full stack developer|               Web|
|           developer|          So

USE RANDOM FOREST TEXT CLASSIFICATION TO CATEGORIZE THE JOB_TITLE COLUMN

In [111]:
from pyspark.ml.feature import CountVectorizer, StringIndexer, IndexToString
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

StatementMeta(Nurture, 1296, 23, Finished, Available)

Encode the column values from categorical to numerical using string indexer, so as to be suitable for running the model

In [112]:
Indexer = StringIndexer(inputCol="job_title_category", outputCol="indexedLabel").fit(df_train2)

StatementMeta(Nurture, 1296, 24, Finished, Available)

Transform the training data with the encoded data

In [113]:
trainingDataIndexed = Indexer.transform(df_train2)

StatementMeta(Nurture, 1296, 25, Finished, Available)

Store the encoded data (indexedLabel) into the labelCopy column created

In [114]:
training_data_indexed = trainingDataIndexed.withColumn("labelCopy", col("indexedLabel"))

StatementMeta(Nurture, 1296, 26, Finished, Available)

Tokenize the training data

In [115]:
tokenizer = Tokenizer(inputCol="job_title", outputCol="words")

StatementMeta(Nurture, 1296, 27, Finished, Available)

Aggregate other variables into a list called features

In [116]:
df_countVectorizer = CountVectorizer(inputCol=tokenizer.getOutputCol(), outputCol="features")

StatementMeta(Nurture, 1296, 28, Finished, Available)

In [117]:
#Initialize the random forest algorithm
df_rf = RandomForestClassifier(labelCol="labelCopy", featuresCol="features")

StatementMeta(Nurture, 1296, 29, Finished, Available)

Convert the encoded column back to the original string format, this works opposite with StringIndexer

In [118]:
#Convert the encoded column back to the original string format, this works opposite with StringIndexer
Converter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=Indexer.labels)

StatementMeta(Nurture, 1296, 30, Finished, Available)

Define a pipeline to pass the preparatory/pre-processing steps

In [119]:
#Define a pipeline to pass the preparatory/pre-processing steps
pipeline = Pipeline(stages=[tokenizer, df_countVectorizer, df_rf, Converter])

StatementMeta(Nurture, 1296, 31, Finished, Available)

In [120]:
#Split the data into training and test set
(train_set, test_set) = training_data_indexed.randomSplit([0.7, 0.3])

StatementMeta(Nurture, 1296, 32, Finished, Available)

Test the model on the test dataset

In [121]:
#Test the model on the test dataset
model = pipeline.fit(train_set)

StatementMeta(Nurture, 1296, 33, Finished, Available)

make predictions of the built model on the test dataset

In [122]:
#make predictions of the built model on the main dataset
predictions_test = model.transform(test_set)
predictions_test.show()

StatementMeta(Nurture, 1296, 34, Finished, Available)

+--------------------+------------------+------------+---------+--------------------+--------------------+--------------------+--------------------+----------+--------------+
|           job_title|job_title_category|indexedLabel|labelCopy|               words|            features|       rawPrediction|         probability|prediction|predictedLabel|
+--------------------+------------------+------------+---------+--------------------+--------------------+--------------------+--------------------+----------+--------------+
|                   2|             Other|         1.0|      1.0|                 [2]|    (377,[24],[1.0])|[9.98862631601492...|[0.49943131580074...|       0.0|      Software|
|          accountant|             Other|         1.0|      1.0|        [accountant]|         (377,[],[])|[9.98862631601492...|[0.49943131580074...|       0.0|      Software|
|       administrator|        Management|         3.0|      3.0|     [administrator]|    (377,[42],[1.0])|[9.43127864576402..

EVALUATING OUR MODEL

In [123]:
evaluator = MulticlassClassificationEvaluator(labelCol="labelCopy", predictionCol="prediction",
                                              metricName="accuracy")

StatementMeta(Nurture, 1296, 35, Finished, Available)

Get the model accuracy

In [124]:
accuracy = evaluator.evaluate(predictions_test)
print("Test Accuracy = %g" % accuracy)

StatementMeta(Nurture, 1296, 36, Finished, Available)

Test Accuracy = 0.520408


In [125]:
predictions2 = model.transform(df_clean2.withColumnRenamed("Candidate_job_title", "job_title"))

StatementMeta(Nurture, 1296, 37, Finished, Available)

In [126]:
predictions2.show()

StatementMeta(Nurture, 1296, 38, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+--------------------+--------------------+--------------------+--------------------+----------+--------------+
|                  ID|      WorkHistory_ID|   Candidate_contact|           job_title|Cnd_start_date|Cnd_end_date|Duration|               words|            features|       rawPrediction|         probability|prediction|predictedLabel|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+--------------------+--------------------+--------------------+--------------------+----------+--------------+
|6ae7162b-104e-eb1...|6ae7162b-104e-eb1...|14c6701e-104e-eb1...|        data analyst|    2019-05-01|  2020-08-01|      16|     [data, analyst]|(377,[9,13],[1.0,...|[6.66505806291916...|[0.33325290314595...|       0.0|      Software|
|44e4bc29-104e-eb1...|44e4bc29-104e-eb1...|03c6701e-104e-eb1...|repo

In [127]:
#Select some columns to track and inspect the predictions
predictions3 = predictions2.select('ID', 'WorkHistory_ID', 'job_title', 'predictedLabel', 'Duration')
predictions3.show()

StatementMeta(Nurture, 1296, 39, Finished, Available)

+--------------------+--------------------+--------------------+--------------+--------+
|                  ID|      WorkHistory_ID|           job_title|predictedLabel|Duration|
+--------------------+--------------------+--------------------+--------------+--------+
|6ae7162b-104e-eb1...|6ae7162b-104e-eb1...|        data analyst|      Software|      16|
|44e4bc29-104e-eb1...|44e4bc29-104e-eb1...|reporting and dat...|      Software|    null|
|71e7162b-104e-eb1...|71e7162b-104e-eb1...|             analyst|      Software|      14|
|76e7162b-104e-eb1...|76e7162b-104e-eb1...|data and process ...|      Software|      19|
|53e4bc29-104e-eb1...|53e4bc29-104e-eb1...|           associate|      Software|      10|
|5de4bc29-104e-eb1...|5de4bc29-104e-eb1...|risk analyst  ris...|      Software|      10|
|7ee7162b-104e-eb1...|7ee7162b-104e-eb1...|analystprogrammer...|      Software|     331|
|9f2bd42f-104e-eb1...|9f2bd42f-104e-eb1...|      risk assistant|      Software|       6|
|a92bd42f-104e-eb1...

Run a logistic regression model

In [128]:
from pyspark.ml.classification import LogisticRegression

StatementMeta(Nurture, 1296, 40, Finished, Available)

In [129]:
#Initialize the logistics regression algorithm
df_lr = LogisticRegression(labelCol="indexedLabel", featuresCol="features")

StatementMeta(Nurture, 1296, 41, Finished, Available)

In [130]:
#Initialize the stages pipeline process
pipeline_lr = Pipeline(stages=[tokenizer, df_countVectorizer, df_lr, Converter])

StatementMeta(Nurture, 1296, 42, Finished, Available)

Train the model

In [131]:
#Train the logistic regression model with the train set
model_lr = pipeline_lr.fit(train_set)

StatementMeta(Nurture, 1296, 43, Finished, Available)

Test the model

In [132]:
#Run predictions on the test set
predictions_test_lr = model_lr.transform(test_set)

StatementMeta(Nurture, 1296, 44, Finished, Available)

Evaluate the model to get the accuracy

In [133]:
#Initialize the MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction",
                                              metricName="accuracy")

StatementMeta(Nurture, 1296, 45, Finished, Available)

In [134]:
#Calculate the accuracy of the model
accuracy_lr = evaluator.evaluate(predictions_test_lr)
print("Test Accuracy (Logistic Regression) = %g" % accuracy_lr)

StatementMeta(Nurture, 1296, 46, Finished, Available)

Test Accuracy (Logistic Regression) = 0.955102


In [135]:
predictions_lr = model_lr.transform(df_clean2.withColumnRenamed("Candidate_job_title", "job_title"))

StatementMeta(Nurture, 1296, 47, Finished, Available)

In [136]:
predictions_lr.show()

StatementMeta(Nurture, 1296, 48, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+--------------------+--------------------+--------------------+--------------------+----------+--------------+
|                  ID|      WorkHistory_ID|   Candidate_contact|           job_title|Cnd_start_date|Cnd_end_date|Duration|               words|            features|       rawPrediction|         probability|prediction|predictedLabel|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+--------------------+--------------------+--------------------+--------------------+----------+--------------+
|6ae7162b-104e-eb1...|6ae7162b-104e-eb1...|14c6701e-104e-eb1...|        data analyst|    2019-05-01|  2020-08-01|      16|     [data, analyst]|(377,[9,13],[1.0,...|[-60.076474454916...|[5.96035675120836...|       5.0|  Data Analyst|
|44e4bc29-104e-eb1...|44e4bc29-104e-eb1...|03c6701e-104e-eb1...|repo

In [137]:
#Track the predictions generated
predictions_lr2 = predictions_lr.select('ID', 'Candidate_contact', 'WorkHistory_ID', 'job_title', 'predictedLabel', 'Duration')
predictions_lr2.show()

StatementMeta(Nurture, 1296, 49, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+--------------+--------+
|                  ID|   Candidate_contact|      WorkHistory_ID|           job_title|predictedLabel|Duration|
+--------------------+--------------------+--------------------+--------------------+--------------+--------+
|6ae7162b-104e-eb1...|14c6701e-104e-eb1...|6ae7162b-104e-eb1...|        data analyst|  Data Analyst|      16|
|44e4bc29-104e-eb1...|03c6701e-104e-eb1...|44e4bc29-104e-eb1...|reporting and dat...|  Data Analyst|    null|
|71e7162b-104e-eb1...|14c6701e-104e-eb1...|71e7162b-104e-eb1...|             analyst|  Data Analyst|      14|
|76e7162b-104e-eb1...|03c6701e-104e-eb1...|76e7162b-104e-eb1...|data and process ...|  Data Analyst|      19|
|53e4bc29-104e-eb1...|14c6701e-104e-eb1...|53e4bc29-104e-eb1...|           associate|         Other|      10|
|5de4bc29-104e-eb1...|14c6701e-104e-eb1...|5de4bc29-104e-eb1...|risk analyst  ris...|  Data Analyst|      10|
|7ee7162b-

**Trying the Decision tree model**

In [138]:
from pyspark.ml.classification import DecisionTreeClassifier

StatementMeta(Nurture, 1296, 50, Finished, Available)

In [139]:
#Initialize the decision tree algorithm
df_dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="features")

StatementMeta(Nurture, 1296, 51, Finished, Available)

In [140]:
#Define the pipeline stages
pipeline_dt = Pipeline(stages=[tokenizer, df_countVectorizer, df_dt, Converter])

StatementMeta(Nurture, 1296, 52, Finished, Available)

In [141]:
#Train the model with the train set
model_dt = pipeline_dt.fit(train_set)

StatementMeta(Nurture, 1296, 53, Finished, Available)

In [142]:
#Deploy the model on the test set
predictions_test_dt = model_dt.transform(test_set)

StatementMeta(Nurture, 1296, 54, Finished, Available)

In [143]:
#Evaluate the model with MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy_dt = evaluator.evaluate(predictions_test_dt)

StatementMeta(Nurture, 1296, 55, Finished, Available)

In [144]:
#Calculate the decision tree accuracy
print("Test Accuracy (Decision Tree) = %g" % accuracy_dt)

StatementMeta(Nurture, 1296, 56, Finished, Available)

Test Accuracy (Decision Tree) = 0.865306


In [145]:
predictions_dt = model_dt.transform(df_clean2.withColumnRenamed("Candidate_job_title", "job_title"))
predictions_dt.show()

StatementMeta(Nurture, 1296, 57, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+--------------------+--------------------+--------------------+--------------------+----------+--------------+
|                  ID|      WorkHistory_ID|   Candidate_contact|           job_title|Cnd_start_date|Cnd_end_date|Duration|               words|            features|       rawPrediction|         probability|prediction|predictedLabel|
+--------------------+--------------------+--------------------+--------------------+--------------+------------+--------+--------------------+--------------------+--------------------+--------------------+----------+--------------+
|6ae7162b-104e-eb1...|6ae7162b-104e-eb1...|14c6701e-104e-eb1...|        data analyst|    2019-05-01|  2020-08-01|      16|     [data, analyst]|(377,[9,13],[1.0,...|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|       5.0|  Data Analyst|
|44e4bc29-104e-eb1...|44e4bc29-104e-eb1...|03c6701e-104e-eb1...|repo

In [146]:
#Track the predictions generated
predictions_dt2 = predictions_dt.select('ID', 'Candidate_contact', 'WorkHistory_ID', 'job_title', 'predictedLabel', 'Duration')
predictions_dt2.show()

StatementMeta(Nurture, 1296, 58, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+--------------+--------+
|                  ID|   Candidate_contact|      WorkHistory_ID|           job_title|predictedLabel|Duration|
+--------------------+--------------------+--------------------+--------------------+--------------+--------+
|6ae7162b-104e-eb1...|14c6701e-104e-eb1...|6ae7162b-104e-eb1...|        data analyst|  Data Analyst|      16|
|44e4bc29-104e-eb1...|03c6701e-104e-eb1...|44e4bc29-104e-eb1...|reporting and dat...|  Data Analyst|    null|
|71e7162b-104e-eb1...|14c6701e-104e-eb1...|71e7162b-104e-eb1...|             analyst|  Data Analyst|      14|
|76e7162b-104e-eb1...|03c6701e-104e-eb1...|76e7162b-104e-eb1...|data and process ...|  Data Analyst|      19|
|53e4bc29-104e-eb1...|14c6701e-104e-eb1...|53e4bc29-104e-eb1...|           associate|         Other|      10|
|5de4bc29-104e-eb1...|14c6701e-104e-eb1...|5de4bc29-104e-eb1...|risk analyst  ris...|  Data Analyst|      10|
|7ee7162b-

In [147]:
candidate_workhist.show()

StatementMeta(Nurture, 1296, 59, Finished, Available)

+--------------------+--------------------+-----------------------+----------------------+--------------------+--------------------------+--------------------+--------------------+
|                  ID|   Candidate_contact|Candidate_workstrt_date|Candidate_workend_date|      WorkHistory_ID|Candidate_work_description| Candidate_job_title|      Cand_workplace|
+--------------------+--------------------+-----------------------+----------------------+--------------------+--------------------------+--------------------+--------------------+
|6ae7162b-104e-eb1...|14c6701e-104e-eb1...|    2019-05-01 00:00:00|   2020-08-01 00:00:00|6ae7162b-104e-eb1...|      ; lendinvest - --...|        Data analyst|Data analyst - Le...|
|44e4bc29-104e-eb1...|03c6701e-104e-eb1...|    2017-01-01 00:00:00|                  null|44e4bc29-104e-eb1...|      key achievements\...|Reporting and Dat...|Reporting and Dat...|
|71e7162b-104e-eb1...|14c6701e-104e-eb1...|    2018-04-01 00:00:00|   2019-05-01 00:00:00|71e71

In [148]:
candidate_skilllevel.show()

StatementMeta(Nurture, 1296, 60, Finished, Available)

+--------------------+--------------------+----------------+----------------+---------------+---------------+--------------------+---------+--------------------+--------------------+----------+
|             Cand_ID| Candidate_contactno|Cand_proficiency|Cand_requirement|Cand_experience|Cand_exp_period|          Cand_skill|Cand_name|  Cand_skilllevel_ID|     Cand_skill_name|Cand_level|
+--------------------+--------------------+----------------+----------------+---------------+---------------+--------------------+---------+--------------------+--------------------+----------+
|ab1e9a1d-104e-eb1...|03c6701e-104e-eb1...|       143570003|       143570000|             43|      143570000|089ac1e6-27bd-ea1...|     null|ab1e9a1d-104e-eb1...|                gdpr|        97|
|ae1e9a1d-104e-eb1...|03c6701e-104e-eb1...|       143570003|       143570000|             43|      143570000|9f9fc31c-8a59-e11...|     null|ae1e9a1d-104e-eb1...|    data warehousing|        99|
|b51e9a1d-104e-eb1...|03c6701e

Join the two tables at their candidate contact

In [149]:
#Merge the two tables at their common column (candidate contact)
join_df = predictions_lr2.join(candidate_skilllevel, predictions_lr2.Candidate_contact == candidate_skilllevel.Candidate_contactno)
join_df.show()

StatementMeta(Nurture, 1296, 61, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+--------------+--------+--------------------+--------------------+----------------+----------------+---------------+---------------+--------------------+---------+--------------------+----------------+----------+
|                  ID|   Candidate_contact|      WorkHistory_ID|           job_title|predictedLabel|Duration|             Cand_ID| Candidate_contactno|Cand_proficiency|Cand_requirement|Cand_experience|Cand_exp_period|          Cand_skill|Cand_name|  Cand_skilllevel_ID| Cand_skill_name|Cand_level|
+--------------------+--------------------+--------------------+--------------------+--------------+--------+--------------------+--------------------+----------------+----------------+---------------+---------------+--------------------+---------+--------------------+----------------+----------+
|7ee7162b-104e-eb1...|03c6701e-104e-eb1...|7ee7162b-104e-eb1...|analystprogrammer...|         Other|     3

In [150]:
#Explore the merged dataframe
new_df = join_df.select('Candidate_contact','Cand_ID', 'job_title', 'predictedLabel', 'Cand_skill_name', 'Cand_experience', 'Cand_level', 'Duration')
new_df.show()

StatementMeta(Nurture, 1296, 62, Finished, Available)

+--------------------+--------------------+--------------------+--------------+----------------+---------------+----------+--------+
|   Candidate_contact|             Cand_ID|           job_title|predictedLabel| Cand_skill_name|Cand_experience|Cand_level|Duration|
+--------------------+--------------------+--------------------+--------------+----------------+---------------+----------+--------+
|03c6701e-104e-eb1...|ab1e9a1d-104e-eb1...|analystprogrammer...|         Other|            gdpr|             43|        97|     331|
|03c6701e-104e-eb1...|ab1e9a1d-104e-eb1...|data and process ...|  Data Analyst|            gdpr|             43|        97|      19|
|03c6701e-104e-eb1...|ab1e9a1d-104e-eb1...|reporting and dat...|  Data Analyst|            gdpr|             43|        97|    null|
|03c6701e-104e-eb1...|ae1e9a1d-104e-eb1...|analystprogrammer...|         Other|data warehousing|             43|        99|     331|
|03c6701e-104e-eb1...|ae1e9a1d-104e-eb1...|data and process ...|  Dat

In [151]:
from pyspark.sql.functions import when, col, lit, regexp_extract

StatementMeta(Nurture, 1296, 63, Finished, Available)

Put the seniority level in a new column

In [152]:
#Create the seniority level column
new_df2 = new_df.withColumn("Seniority",
                   when(col("job_title").rlike("(?i)senior"), "Senior")
                   .when(col("job_title").rlike("(?i)junior"), "Junior")
                   .when(col("job_title").rlike("(?i)mid"), "Mid")
                   .when(col("job_title").rlike("(?i)head"), "Head")
                   .when(col("job_title").rlike("(?i)lead"), "Lead")
                   .otherwise("None"))


StatementMeta(Nurture, 1296, 64, Finished, Available)

In [153]:
new_df2.show(truncate = False)

StatementMeta(Nurture, 1296, 65, Finished, Available)

+------------------------------------+------------------------------------+-----------------------------+--------------+----------------+---------------+----------+--------+---------+
|Candidate_contact                   |Cand_ID                             |job_title                    |predictedLabel|Cand_skill_name |Cand_experience|Cand_level|Duration|Seniority|
+------------------------------------+------------------------------------+-----------------------------+--------------+----------------+---------------+----------+--------+---------+
|03c6701e-104e-eb11-a812-000d3a8717e3|ab1e9a1d-104e-eb11-a812-00224800b0a0|analystprogrammer  programmer|Other         |gdpr            |43             |97        |331     |None     |
|03c6701e-104e-eb11-a812-000d3a8717e3|ab1e9a1d-104e-eb11-a812-00224800b0a0|data and process analyst     |Data Analyst  |gdpr            |43             |97        |19      |None     |
|03c6701e-104e-eb11-a812-000d3a8717e3|ab1e9a1d-104e-eb11-a812-00224800b0a0|repor

Aggregate for each job_category

In [154]:
#Aggregate for each job_category
grouped_new = new_df2.groupBy('Candidate_contact','predictedLabel', 'job_title', 'Cand_skill_name', 'Cand_experience', 'Cand_level', 'Duration', 'Seniority').count().sort('predictedLabel', ascending=False)
grouped_new.show()

StatementMeta(Nurture, 1296, 66, Finished, Available)

+--------------------+--------------+--------------------+--------------------+---------------+----------+--------+---------+-----+
|   Candidate_contact|predictedLabel|           job_title|     Cand_skill_name|Cand_experience|Cand_level|Duration|Seniority|count|
+--------------------+--------------+--------------------+--------------------+---------------+----------+--------+---------+-----+
|e2a9ca41-4a65-eb1...|           Web|       web developer|          postgresql|              0|        13|       5|     None|    1|
|a6367b21-fe69-eb1...|           Web|  lead web developer|sql server integr...|            134|        67|       8|     Lead|    1|
|b8fcacdf-6265-eb1...|           Web|web administrator...|   microsoft asp.net|              5|        13|      26|     None|    1|
|07a7700c-4c61-eb1...|           Web|       web developer|java 2 enterprise...|              9|        45|      41|     None|    1|
|a6367b21-fe69-eb1...|           Web|       web developer|                ss

Aggregate for each ID

In [155]:
#Aggregate for each candidate contact ID
grouped_ID = new_df2.groupBy('Candidate_contact','Cand_ID', 'job_title', 'predictedLabel', 'Cand_skill_name', 'Cand_experience', 'Cand_level', 'Duration', 'Seniority').count().sort('Cand_ID', ascending=False)
grouped_ID.show()

StatementMeta(Nurture, 1296, 67, Finished, Available)

+--------------------+--------------------+--------------------+--------------+------------------+---------------+----------+--------+---------+-----+
|   Candidate_contact|             Cand_ID|           job_title|predictedLabel|   Cand_skill_name|Cand_experience|Cand_level|Duration|Seniority|count|
+--------------------+--------------------+--------------------+--------------+------------------+---------------+----------+--------+---------+-----+
|4a95b4fe-6961-eb1...|ffffd810-6a61-eb1...|business data ana...|  Data Analyst|project management|              0|        13|      13|     None|    1|
|4a95b4fe-6961-eb1...|ffffd810-6a61-eb1...|senior business i...|         Other|project management|              0|        13|      31|   Senior|    1|
|4a95b4fe-6961-eb1...|ffffd810-6a61-eb1...|profit and loss a...|  Data Analyst|project management|              0|        13|      21|     None|    1|
|4a95b4fe-6961-eb1...|ffffd810-6a61-eb1...|           assistant|         Other|project managem

In [156]:
#Aggregate for each candidate contact ID
grouped_ID = new_df2.groupBy('Candidate_contact','Cand_ID', 'job_title', 'predictedLabel', 'Cand_skill_name', 'Cand_experience', 'Cand_level','Duration', 'Seniority').count().sort('Cand_ID', ascending=False)
grouped_ID.show(100,truncate = False)

StatementMeta(Nurture, 1296, 68, Finished, Available)

+------------------------------------+------------------------------------+---------------------------------------------------------------+---------------+--------------------------------------+---------------+----------+--------+---------+-----+
|Candidate_contact                   |Cand_ID                             |job_title                                                      |predictedLabel |Cand_skill_name                       |Cand_experience|Cand_level|Duration|Seniority|count|
+------------------------------------+------------------------------------+---------------------------------------------------------------+---------------+--------------------------------------+---------------+----------+--------+---------+-----+
|4a95b4fe-6961-eb11-a812-0022481a6ace|ffffd810-6a61-eb11-a812-0022481a6ace|profit and loss analyst                                        |Data Analyst   |project management                    |0              |13        |21      |None     |1    |
|4a95b4fe-69

In [157]:
candidateID = '4a95b4fe-6961-eb11-a812-0022481a6ace'

StatementMeta(Nurture, 1296, 69, Finished, Available)

In [158]:
#Get the profile of the candidate ID defined
df_ID = grouped_ID.filter(grouped_ID.Candidate_contact == candidateID)
df_ID.show(50, truncate=False)

StatementMeta(Nurture, 1296, 70, Finished, Available)

+------------------------------------+------------------------------------+--------------------------------------------------+--------------+------------------+---------------+----------+--------+---------+-----+
|Candidate_contact                   |Cand_ID                             |job_title                                         |predictedLabel|Cand_skill_name   |Cand_experience|Cand_level|Duration|Seniority|count|
+------------------------------------+------------------------------------+--------------------------------------------------+--------------+------------------+---------------+----------+--------+---------+-----+
|4a95b4fe-6961-eb11-a812-0022481a6ace|ffffd810-6a61-eb11-a812-0022481a6ace|profit and loss analyst                           |Data Analyst  |project management|0              |13        |13      |None     |1    |
|4a95b4fe-6961-eb11-a812-0022481a6ace|ffffd810-6a61-eb11-a812-0022481a6ace|accounts receivablepayable clerk                  |Other         |project

***EXTRACT SKILLS FROM A CV***

In [159]:
cv = """
PERSONAL PROFILE
Personal Statement
A highly focused Data Analyst undergoing a master's in Data Science seeks a challenging position with your
organization that will enable him to capitalize on sound technical expertise with opportunities for personal and
professional growth.
EDUCATION AND TRAINING
Masters of Science
University of Salford [ 16 Jan 2023 – 30 Apr 2024 ]
Address: The Crescent Salford, M5 4WT Greater Manchester (United Kingdom)
Website: www.salford.ac.uk
Field(s) of study: Data Science
Masters of Engineering
University of Leeds [ 19 Sep 2017 – 27 Feb 2019 ]
Address: Buslingthorpe Lane, LS7 2DG Leeds
Website: www.leeds.ac.uk
Field(s) of study: Electronics and Electrical Engineering
Bachelor of Technology
Rivers State University of Science and Technology [ 7 Oct 2008 – 28 Aug 2014 ]
Address: PMB 5080 Port Harcourt (Nigeria)
Website: www.ust.edu.ng
Field(s) of study: Electrical Engineering
Final grade: First Class
WORK EXPERIENCE
Database Developer
Cracking Good Food C.I.C. [ 18 Jun 2023 – Current ]
City: Manchester
Country: United Kingdom
• Designed and developed a comprehensive, user-friendly database system for managing all aspects of the
business.
• Implemented advanced data analysis functionality and reporting capabilities to improve day-to-day
efficiency.
• Assisted in the seamless transfer of raw data from existing sources to the new database.
• Conducted training sessions to educate users on database functions and ensure effective utilization.
• Demonstrated exceptional organizational skills in managing multiple tasks within deadlines.
• Motivated by the organization's mission, paid close attention to detail in project management, design, and
communications.
IT Support Assistant (Voluntary)
SOVEREIGN HOUSE GH [ 23 May 2023 – Current ]
City: Salford
Dagogo Godwin Orifama
Gender: Male  Phone number: (+44) 7425363844  Email address: dagoris2010@gmail.com
 LinkedIn: https://www.linkedin.com/in/dagogoorifama/
 Home: 9 Co-Operative Street, M6 5DW Salford (United Kingdom)
Country: United Kingdom
• Provided essential computer coding skills and comprehensive IT support to children in the Salford
community, fostering their self-esteem and enhancing their life chances.
• Taught and guided children on fundamental concepts such as sequences, variables, algorithms, HTML, CSS,
JavaScript, coding, and e-safety.
• Expanded the curriculum beyond standard coding education by introducing game development, database
management, and Python programming, equipping children with versatile technical skills and promoting
critical thinking and problem-solving abilities.
• Collaborated with an enthusiastic team to create a supportive learning environment, where students' talents
were nurtured and their potential maximized.
• Made a positive impact on children's lives by volunteering multiple evenings per week and witnessing their
growth and development in both technical proficiency and self-confidence.
Regional Financial Analyst
CJC Markets Nigeria [ 16 Jun 2022 – 16 Dec 2022 ]
City: Port Harcourt
Country: Nigeria
Website: https://cjcmarketsglobal.com/
• Reviewed statistical information to determine financial trends for use in investment planning
• Completed thorough research into the financial exchange market and reported on relevant trends.
• Reviewed monthly requisitions for accuracy and completeness, reconciled transactions and determining
payment approval statuses.
• Delivered financial oversight and provided detailed training to corporate clients.
• Developed forecasting tools to analyze the financial exchange market for clients.
• Worked with regional teams leads to identify issues affecting financial results and profitability.
• Reviewed financial reports and recommended strategies to streamline operations, increase productivity and
grow company profits.
NETWORKS AND MEMBERSHIPS
Graduate Member
[ 18 Jun 2020 – Current ]
The Nigerian Society of Engineers
Graduate Member
[ Sep 2017 – Current ]
Institute of Electrical and Electronics Engineers, IEEE
HOBBIES AND INTERESTS
Musics, Movies and Games
Karate Player
RECOMMENDATIONS
Provided on request.

"""

StatementMeta(Nurture, 1296, 71, Finished, Available)

In [160]:
cv

StatementMeta(Nurture, 1296, 72, Finished, Available)

"\nPERSONAL PROFILE\nPersonal Statement\nA highly focused Data Analyst undergoing a master's in Data Science seeks a challenging position with your\norganization that will enable him to capitalize on sound technical expertise with opportunities for personal and\nprofessional growth.\nEDUCATION AND TRAINING\nMasters of Science\nUniversity of Salford [ 16 Jan 2023 – 30 Apr 2024 ]\nAddress: The Crescent Salford, M5 4WT Greater Manchester (United Kingdom)\nWebsite: www.salford.ac.uk\nField(s) of study: Data Science\nMasters of Engineering\nUniversity of Leeds [ 19 Sep 2017 – 27 Feb 2019 ]\nAddress: Buslingthorpe Lane, LS7 2DG Leeds\nWebsite: www.leeds.ac.uk\nField(s) of study: Electronics and Electrical Engineering\nBachelor of Technology\nRivers State University of Science and Technology [ 7 Oct 2008 – 28 Aug 2014 ]\nAddress: PMB 5080 Port Harcourt (Nigeria)\nWebsite: www.ust.edu.ng\nField(s) of study: Electrical Engineering\nFinal grade: First Class\nWORK EXPERIENCE\nDatabase Developer\n

In [161]:
import re

StatementMeta(Nurture, 1296, 73, Finished, Available)

In [162]:
#Create a function to preprocess the cv text
def resume_skills(text, skill_list):
    skill = []
    
    for Candskill in skill_list:
        pattern = r"\b{}\b".format(re.escape(Candskill))
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            skill.append(Candskill)
    return skill

StatementMeta(Nurture, 1296, 74, Finished, Available)

**Edensmith's skills list library**

In [163]:
crimson_skills = spark.read.load('abfss://enriched@esgadls2.dfs.core.windows.net/nurture/skills/skills_database/v1', format='delta'

)

crimson_skills.show(10)

crimson_skills=crimson_skills.select(lower(col('crimson_name')))

crimson_skills.show(10)

skill_exists=crimson_skills.rdd.map(lambda x: x[0]).collect()

StatementMeta(Nurture, 1296, 75, Finished, Available)

+----------------+--------+----------+
|    crimson_name|JobTitle|All_skills|
+----------------+--------+----------+
|          Python|    null|      null|
|        Power BI|    null|      null|
|         PowerBI|    null|      null|
|  Power Platform|    null|      null|
|   Amazon Athena|    null|      null|
|Data Engineering|    null|      null|
|      Amazon EMR|    null|      null|
|             Git|    null|      null|
|      AWS Lambda|    null|      null|
|      Amazon EC2|    null|      null|
+----------------+--------+----------+
only showing top 10 rows

+-------------------+
|lower(crimson_name)|
+-------------------+
|             python|
|           power bi|
|            powerbi|
|     power platform|
|      amazon athena|
|   data engineering|
|         amazon emr|
|                git|
|         aws lambda|
|         amazon ec2|
+-------------------+
only showing top 10 rows



Extract the skills from the cv

In [164]:
if __name__ == '__main__':
    text = cv
    
    skill_list = skill_exists
    
    extracted_skills = resume_skills(text, skill_list)
    
    if extracted_skills:
        print('Skills_extracted', extracted_skills)
    else:
            print('No skills matched')

StatementMeta(Nurture, 1296, 76, Finished, Available)

Skills_extracted ['python', 'data analysis', 'data science', 'algorithms', 'streamline', 'society of engineers', 'request', 'python programming', 'project management', 'project management', 'master', 'master', ' engineering', ' engineering', ' engineering', ' engineering', 'javascript', ' science and technology', 'html', 'forecasting', 'forecasting', 'communications', 'aspects', 'approval', 'a']


In [165]:
##Convert the skills to lowercase
extracted_skills2 = [skill.lower() for skill in extracted_skills]
print(extracted_skills2)

StatementMeta(Nurture, 1296, 77, Finished, Available)

['python', 'data analysis', 'data science', 'algorithms', 'streamline', 'society of engineers', 'request', 'python programming', 'project management', 'project management', 'master', 'master', ' engineering', ' engineering', ' engineering', ' engineering', 'javascript', ' science and technology', 'html', 'forecasting', 'forecasting', 'communications', 'aspects', 'approval', 'a']


In [166]:
from pyspark.sql.functions import sum

StatementMeta(Nurture, 1296, 78, Finished, Available)

**RECOMMENDER SYSTEM**

**COSINE SIMILARITY**

In [167]:
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import split, col
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import SparseVector
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import array, udf
from pyspark.ml.linalg import SparseVector, DenseVector, Vectors
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.sql.types import IntegerType, ArrayType, StringType
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

StatementMeta(Nurture, 1296, 79, Finished, Available)

In [168]:
grouped_ID_pd = grouped_ID.toPandas()
grouped_ID_pd

StatementMeta(Nurture, 1296, 80, Finished, Available)

,Candidate_contact,Cand_ID,job_title,predictedLabel,Cand_skill_name,Cand_experience,Cand_level,Duration,Seniority,count
0,4a95b4fe-6961-eb11-a812-0022481a6ace,ffffd810-6a61-eb11-a812-0022481a6ace,data analyst quality and performance,Data Analyst,project management,0.0,13.0,NaN,None,1
1,4a95b4fe-6961-eb11-a812-0022481a6ace,ffffd810-6a61-eb11-a812-0022481a6ace,profit and loss analyst,Data Analyst,project management,0.0,13.0,21.0,None,1
2,4a95b4fe-6961-eb11-a812-0022481a6ace,ffffd810-6a61-eb11-a812-0022481a6ace,product controller,Software,project management,0.0,13.0,5.0,None,1
3,4a95b4fe-6961-eb11-a812-0022481a6ace,ffffd810-6a61-eb11-a812-0022481a6ace,revenue accountant,Other,project management,0.0,13.0,28.0,None,1
4,4a95b4fe-6961-eb11-a812-0022481a6ace,ffffd810-6a61-eb11-a812-0022481a6ace,senior business intelligence consultant,Other,project management,0.0,13.0,31.0,Senior,1
...,...,...,...,...,...,...,...,...,...,...
1642867,81b013fa-e629-ed11-9db2-0022481b8794,0000359f-a22a-ed11-9db2-0022481b8b3b,business objects platinum partner,Other,tm1,11.0,45.0,22.0,None,1
1642868,81b013fa-e629-ed11-9db2-0022481b8794,0000359f-a22a-ed11-9db2-0022481b8b3b,enterprise information architect head of bi,Other,tm1,11.0,45.0,NaN,Head,1
1642869,81b013fa-e629-ed11-9db2-0022481b8794,0000359f-a22a-ed11-9db2-0022481b8b3b,sap business objects business intelligence con...,Software,tm1,11.0,45.0,9.0,None,1
1642870,81b013fa-e629-ed11-9db2-0022481b8794,0000359f-a22a-ed11-9db2-0022481b8b3b,business intelligence analytics consultant,Software,tm1,11.0,45.0,6.0,None,1


In [169]:
#Treat null values
data = grouped_ID_pd.dropna(subset=['Cand_skill_name'])
data2 = data.dropna(subset=['predictedLabel'])

StatementMeta(Nurture, 1296, 81, Finished, Available)

In [170]:
#Join the job category and the the skills for that category, such that for each job category, all the skills will be assiciated to it.
grouped_ID_pd2 = data2.groupby('predictedLabel')['Cand_skill_name'].apply(','.join).reset_index()

StatementMeta(Nurture, 1296, 82, Finished, Available)

In [171]:
#Embed the resulting dataframe using the TF-IDF vectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(grouped_ID_pd2['Cand_skill_name'])

StatementMeta(Nurture, 1296, 83, Finished, Available)

In [172]:
#Initialize the cosine similarity
cosSim = linear_kernel(tfidf_matrix, tfidf_matrix)

StatementMeta(Nurture, 1296, 84, Finished, Available)

In [173]:
#Attach an index value to each vector feature as a unique identifier
indices2 = pd.Series(grouped_ID_pd2.index, index=grouped_ID_pd2['predictedLabel']).drop_duplicates()

StatementMeta(Nurture, 1296, 85, Finished, Available)

In [174]:
indices2

StatementMeta(Nurture, 1296, 86, Finished, Available)

predictedLabel
Applied Science     0
Data Analyst        1
Data Architect      2
Data Engineer       3
Data Management     4
Data Science        5
Engineering         6
Management          7
Operations          8
Other               9
Software           10
Web                11
dtype: int64

In [175]:
grouped_ID_pd2.head()

StatementMeta(Nurture, 1296, 87, Finished, Available)

,predictedLabel,Cand_skill_name
0,Applied Science,"power bi,power bi,natural language processing,..."
1,Data Analyst,"project management,project management,project ..."
2,Data Architect,"data warehousing,data warehousing,data warehou..."
3,Data Engineer,"tableau,microsoft transact-sql,data warehousin..."
4,Data Management,"ole,ole,bachelors degree > bachelor of science..."


In [176]:
#Define the Cosine similarity recommender system while calculating the cosine similarity between the two careers
def recommender_system(current_job_title, target_job_title):
    
    # Get the index of the current job title and target job title
    idx_current = indices2[current_job_title]
    idx_target = indices2[target_job_title]
    
    # Calculate the similarity scores for the current job title
    sim_scores = list(enumerate(cosSim[idx_current]))
    
    # Sort the job titles based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the most similar job titles
    sim_scores = sim_scores[1:6]
    
    # Get the job title indices
    job_indices = [i[0] for i in sim_scores]
    
    # Extract the skills of the most similar job titles and the target job
    current_skills = set(grouped_ID_pd2['Cand_skill_name'].iloc[idx_current].split(','))
    target_skills = set(grouped_ID_pd2['Cand_skill_name'].iloc[idx_target].split(','))
    
    # Calculate the missing skills
    missing_skills = target_skills - current_skills
    
    return missing_skills

StatementMeta(Nurture, 1296, 88, Finished, Available)

In [177]:
#Deploy the recommender model on the current and target job
current_job = "Data Science"
target_job = "Data Engineer"
print(recommender_system(current_job, target_job))

StatementMeta(Nurture, 1296, 89, Finished, Available)

{'sonic esb', 'sap apo technical skills', 'neolane', 'oracle standard benefits', 'shona', 'microsoft certified product specialist', 'sap sd md', 'institute of leadership & management', 'noc', 'catv', 'oracle sqlnet', 'sun hardware > sun fire servers', 'ecosystems', 'leveraging resources', 'cat 5 ethernet', 'token ring', 'sap wm', 'sap sd sales', 'sage business management software > sage 50 suite', 'icm qualification', 'appc', 'reuters kondor+', 'jdedwards suite', 'retek', 'veritas', 'security clearance > security clearance ctc', 'system implementation', 'cae', 'sage accounting software > sage line 500', 'pegasus opera', 'cosmos', 'progress 4gl', 'investment administration qualification', 'external reporting', 'petty cash', 'o/a levels/grades', 'royal astronomical society', 'uniface', 'hyperion enterprise', 'pdms', 'altova > altova xmlspy', 'computer associates unicenter ca-7', 'electrical engineer', 'lec', 'sybase replication server', 'bachelors degree > bachelor of informatics', 'micr

In [178]:
#Get the recommendations
recommender_system(current_job, target_job)

StatementMeta(Nurture, 1296, 90, Finished, Available)

{'sonic esb',
 'sap apo technical skills',
 'neolane',
 'oracle standard benefits',
 'shona',
 'microsoft certified product specialist',
 'sap sd md',
 'institute of leadership & management',
 'noc',
 'catv',
 'oracle sqlnet',
 'sun hardware > sun fire servers',
 'ecosystems',
 'leveraging resources',
 'cat 5 ethernet',
 'token ring',
 'sap wm',
 'sap sd sales',
 'sage business management software > sage 50 suite',
 'icm qualification',
 'appc',
 'reuters kondor+',
 'jdedwards suite',
 'retek',
 'veritas',
 'security clearance > security clearance ctc',
 'system implementation',
 'cae',
 'sage accounting software > sage line 500',
 'pegasus opera',
 'cosmos',
 'progress 4gl',
 'investment administration qualification',
 'external reporting',
 'petty cash',
 'o/a levels/grades',
 'royal astronomical society',
 'uniface',
 'hyperion enterprise',
 'pdms',
 'altova > altova xmlspy',
 'computer associates unicenter ca-7',
 'electrical engineer',
 'lec',
 'sybase replication server',
 'bache